- Importing the libraries here...

In [2]:
import nltk
import pandas as pd
import spacy
import en_core_web_lg
nlp = en_core_web_lg.load()
import re
import truecase
import numpy as np
import ast
import json

In [3]:
tech = pd.read_csv("Eng_pydictionary_2.csv")
summary = pd.read_csv("1000_tagged_simanchala/1000_Simanchala_Manual_Tagged_final.csv")
output = pd.read_csv("Dataset/output - output.csv")

In [4]:
summary = summary.loc[:1]

In [5]:
len(tech)

5291

In [6]:
summary = summary.sort_values(by=['id'], ascending=False)
summary.reset_index(inplace = True, drop = True)

In [7]:
summary.drop(columns = ["Note"], axis =1, inplace = True)

In [8]:
summary.head()

,id,summaries,exact_matched_patt_contextual
0,-4ldAQ53se0hp9GU5FgnIQ_0000,an extremely driven professional within the bi...,NaN
1,-18msGiQiE2XLdy6XWGAEQ_0000,10+ years of leveraged finance and investment ...,NaN


In [9]:
tech.head()

,Unnamed: 0,English_word
0,0,Lattice
1,16,EMERGE Background Checks
2,35,Tenfold
3,39,Friday Feedback
4,45,Data Dwell


In [10]:

tech.columns = ["index", "title"]
tech.drop(columns = ["index"], axis = 1, inplace=True)

In [11]:
tech.reset_index(inplace = True, drop = True)

In [12]:
# for i in range(0, len(summary)):
#     if summary["summaries"][i] is not np.nan:
#         demo = summary["summaries"][i]
#         res = demo.strip('][')
#         resl = []
#         resl.append(res)
#         summary["summaries"][i] = resl
        
#     else:
#         summary["summaries"][i] = '[]'
#         demo = summary["summaries"][i]
#         res = demo.strip('][')
#         resl = []
#         resl.append(res)
#         summary["summaries"][i] = resl

# converts the string of list of string
def summary_text_preprocess(df, summary_col_name):
    
    for i in range(0, len(df)):
        
        if df[summary_col_name][i] is not np.nan:
            demo = df[summary_col_name][i]
            res = demo.strip('][')
            resl = []
            resl.append(res)
            df[summary_col_name][i] = resl

        else:
            df[summary_col_name][i] = '[]'
            demo = df[summary_col_name][i]
            res = demo.strip('][')
            resl = []
            resl.append(res)
            df[summary_col_name][i] = resl

    return df

In [13]:
summary = summary_text_preprocess(summary, "summaries")

# Truecasing

In [14]:
for ind in summary.index:
    summary["summaries"][ind][0] = truecase.get_true_case(summary["summaries"][ind][0])

In [15]:

def dictionary_with_title_description(df):
    company_names_description = {}
    
    for i in range(0, len(df)):
        
        doc = nlp(df.loc[i, "summaries"][0])
        
        for X in doc.ents:
            if (X.label_ == 'ORG'):
                
                company_names_description.setdefault(df.loc[i, "id"], [])
                company_names_description[df.loc[i, "id"]].append(X.text)
                
    print("Company names extracted from Summaries : \n",company_names_description)

    return company_names_description

In [16]:
# function call
company_names_description = dictionary_with_title_description(summary)

Company names extracted from Summaries : 
 {'-4ldAQ53se0hp9GU5FgnIQ_0000': ['Biopharmeceutical', 'Biopharma', 'Rabs', 'Isolator', 'Biologics', 'GMP', 'GMP', 'Tallaght', 'Ehs ... Institute', 'BSC', 'Alexion pharmaceuticals, Inc.', 'Alexion pharmaceuticals, Inc.', 'Alexion pharmaceuticals, Inc.', 'Alexion pharmaceuticals, Inc.'], '-18msGiQiE2XLdy6XWGAEQ_0000': ['Finance', 'Clos', 'Cdos', 'Co-Investments', 'Buyside', 'Multi', 'Clos', 'Cdos', 'Healthcare', 'CRE', 'Finance', 'Clos', 'Cdos', 'Co-Investments', 'Buyside', 'Multi', 'Clos', 'Cdos', 'Healthcare', 'CRE', 'CIT Healthcare']}


In [18]:
# taking unique and removing duplicate names
for key_3, value_3 in company_names_description.items():
    company_names_description[key_3] = list(set(company_names_description[key_3]))

In [19]:
def new_swapped_dict(df, company_names_description):

    
    final_dictionary = {}
    
    for key_2, val_2 in company_names_description.items():
        
        for m in range(0 , len(tech)):
            for o in range(0, len(val_2)):
                b = tech.loc[m, "title"]
                b=b.lower()
                company_names_description[key_2][o] = company_names_description[key_2][o].lower()
                
                if company_names_description[key_2][o] == b:
                    final_dictionary.setdefault(key_2,[])
                    final_dictionary[key_2].append(b)
                
    return final_dictionary

In [20]:
# function call
final_dictionary = new_swapped_dict(summary, company_names_description)

In [21]:
final_dictionary

{}

In [22]:
tech["id"] = tech.index
tech.head()

,title,id
0,Lattice,0
1,EMERGE Background Checks,1
2,Tenfold,2
3,Friday Feedback,3
4,Data Dwell,4


In [23]:
try:
  
  for k, v in final_dictionary.items():
    # x = summary["summaries"].loc(summary["id"] == k)
    # print(x)
    new_row = {'id':k,'Technology_Names': final_dictionary[k]}
    output = output.append(new_row, ignore_index=True)

except Exception as e :
    print("\n\n", e)

In [26]:
## Extracting summary to Output sheet
for ind in output.index:
    for i in summary.index:
        
        if output["id"][ind]==summary["id"][i]:
            output["summaries"][ind] = summary["summaries"][i]

In [28]:
output.to_csv("NER_train_lg_moreTAGS_simple_spacy.csv", index = False)

In [29]:
df_summary = pd.read_csv("NER_train_lg_moreTAGS_simple_spacy.csv")
df_summary.head()

,id,summaries,Technology_Names


In [30]:
# Lowercasing the summary is important for exact match finding
for ind in output.index:
    output["summaries"][ind][0] = output["summaries"][ind][0].lower()

In [31]:
output.head()

,id,summaries,Technology_Names


In [32]:
df_summary.head()

,id,summaries,Technology_Names


In [33]:
# Applying Regex to filter out Invalid Words

def get_exact_match(txt, tech):
    try:
        patt = '|'.join(['\\b'+elem+'\\b' for elem in tech])
        matched_patt = re.findall(patt,txt[0])
    except Exception as e:
        print(e)
        matched_patt = []
    return matched_patt

In [34]:
output['matched_NER_by_borders'] = output.apply(lambda x: get_exact_match(x["summaries"], x["Technology_Names"]), axis = 1)

ValueError: Wrong number of items passed 3, placement implies 1

In [35]:
output.head()

,id,summaries,Technology_Names


In [57]:
# Applying Regex to filter out Invalid Words

def get_exact_match_2(txt, tech):
    try:
        patt = '|'.join(['\\s'+elem+'\\s' for elem in tech])
        matched_patt = re.findall(patt,txt[0])
    except Exception as e:
        print(e)
        matched_patt = []
    return matched_patt

In [58]:
output['matched_NER_final'] = output.apply(lambda x: get_exact_match_2(x["summaries"], x["matched_NER_by_borders"]), axis = 1)

In [59]:
# output.to_csv("Train_Data/Accuracy_Data/Dataset_to_run_on_all/Output/NER_tagged_data_ner_spacy_own_vocab_100_regex_final.csv", index = False)

In [60]:
# strip by whitespace
for i in range(0,len(output)):
    for j in range(0, len(output['matched_NER_final'][i])):
        output['matched_NER_final'][i][j] = output['matched_NER_final'][i][j].strip()
#     print(output['matched_NER_final'][i])
    

In [61]:
df_company_names_description = pd.DataFrame(list(company_names_description.items()))
df_company_names_description.columns = ["id","NER_Extracted_Tags"]

In [62]:
# df_company_names_description["summaries"] = np.nan

In [63]:
# # Extracting summary to Output sheet
# for ind in df_company_names_description.index:
#     for i in df_summary.index:
        
#         if df_company_names_description["id"][ind]==df_summary["id"][i]:
#             df_company_names_description["summaries"][ind] = df_summary["summaries"][i]

In [64]:
df_company_names_description.head()
# this represents company names extracted in Round 1 from spacy NER

,id,NER_Extracted_Tags
0,zsDSYc5UzpyXekOABZBfnQ_0000,[american red cross]
1,zs1YNrrCeorFkGLEXSSp-A_0000,"[ibm, department of n. c. state, department of..."
2,zowirq8ZhxhchApaRMhNSA_0000,"[linkedin, international trade, export-import ..."
3,zoQVkPCfXB9n3AaPGHHwzg_0000,"[& entrepreneurship, european& international]"
4,zmIr2glBZ3Ef8CSS0jw1og_0000,"[multicultural and multi-functional, volvo, co..."


In [65]:
output.head()
# output is final dictionary basically

,id,summaries,Technology_Names,matched_NER_by_borders,matched_NER_final
0,zekuTL0ONYTX2XawUBa-9A_0000,[i am a self motivated professional with 13+ y...,[dell],"[dell, dell, dell, dell, dell]",[]
1,z5LcE0BcZevf5WmwL8nunQ_0000,"[bachelor of science( b. s) degree in"" telecom...",[juniper],"[juniper, juniper, juniper]",[]
2,xisjoyR4S9lY8F6ukYJKCA_0000,"[highly organized, creative, want to be involv...",[adobe],[adobe],[]
3,udzSvEueVZeZraRcjx0Uvw_0000,"[at javelin engineering, our manufacturing goa...",[assembly],"[assembly, assembly, assembly, assembly, assem...","[assembly, assembly, assembly, assembly]"
4,uYqmEeeHOKc8PLfTA7SwBg_0000,"[producer, director, video editor, video shoot...",[adobe],"[adobe, adobe, adobe, adobe]","[adobe, adobe, adobe, adobe]"


In [66]:
len(df_company_names_description)

979

In [67]:
len(output)

88

In [68]:
# df_company_names_description = df_company_names_description.sort_values(by=['id'], ascending=False)

In [69]:
# df_company_names_description.to_csv("Dataset/Output/NER-output/NER_Extracted_Entities__lg_truecase_moreTAGS.csv", index = False)

In [70]:
# list entries are there so merge will not happend, converting into tuple
# df_company_names_description.b = df_company_names_description.b.apply(tuple)

In [71]:
output = output.merge(df_company_names_description, 'left')

In [72]:
# output = pd.merge(output, df_company_names_description,on = ["id"], how = 'left')

In [73]:
len(output)

88

In [74]:
orig_df = summary.loc[:,["id", "summaries", "exact_matched_patt_contextual"]]

In [75]:
df_final = orig_df.merge(output, 'left', on = ["id"])

In [76]:
# orig_df = summary.loc[:,["id", "summaries", "exact_matched_patt_contextual"]]
# df_final = orig_df.merge(output, 'left', on = ["id"])
df_final.drop(columns = "summaries_y", axis  = 1, inplace = True)

In [77]:
df_final.head()

,id,summaries_x,exact_matched_patt_contextual,Technology_Names,matched_NER_by_borders,matched_NER_final,NER_Extracted_Tags
0,zsDSYc5UzpyXekOABZBfnQ_0000,[Senior director of clinical services housing ...,NaN,NaN,NaN,NaN,NaN
1,zs1YNrrCeorFkGLEXSSp-A_0000,[I believe that passions are meant to be share...,NaN,NaN,NaN,NaN,NaN
2,zowirq8ZhxhchApaRMhNSA_0000,[Bill Bryant is founder and chairman of Bryant...,NaN,NaN,NaN,NaN,NaN
3,zoQVkPCfXB9n3AaPGHHwzg_0000,[Undertaking a Trilingual masters degree in Eu...,NaN,NaN,NaN,NaN,NaN
4,zmIr2glBZ3Ef8CSS0jw1og_0000,[Career objective a role within marketing and ...,NaN,NaN,NaN,NaN,NaN


In [78]:
# # TrueCase summary copy from saved df_summary to df_final summary column for matching id's
# for i in df_final.index:
#     df_final["summaries_x"][i] =df_summary["summaries"].loc[df_summary["id"]==df_final["id"][i]]

In [79]:
for i in df_final.index:
    
    for j in df_summary.index:
        if df_final["id"][i]==df_summary["id"][j]:
            df_final["summaries_x"][i] = df_summary["summaries"][j]

In [80]:
df_final.head()

,id,summaries_x,exact_matched_patt_contextual,Technology_Names,matched_NER_by_borders,matched_NER_final,NER_Extracted_Tags
0,zsDSYc5UzpyXekOABZBfnQ_0000,[Senior director of clinical services housing ...,NaN,NaN,NaN,NaN,NaN
1,zs1YNrrCeorFkGLEXSSp-A_0000,[I believe that passions are meant to be share...,NaN,NaN,NaN,NaN,NaN
2,zowirq8ZhxhchApaRMhNSA_0000,[Bill Bryant is founder and chairman of Bryant...,NaN,NaN,NaN,NaN,NaN
3,zoQVkPCfXB9n3AaPGHHwzg_0000,[Undertaking a Trilingual masters degree in Eu...,NaN,NaN,NaN,NaN,NaN
4,zmIr2glBZ3Ef8CSS0jw1og_0000,[Career objective a role within marketing and ...,NaN,NaN,NaN,NaN,NaN


In [81]:
len(df_final)

1001

In [82]:
df_final["id"].nunique()

1001

In [83]:
df_final.rename(columns = {"summaries_x":"summaries"}, inplace = True)

In [84]:
df_final.head()

,id,summaries,exact_matched_patt_contextual,Technology_Names,matched_NER_by_borders,matched_NER_final,NER_Extracted_Tags
0,zsDSYc5UzpyXekOABZBfnQ_0000,[Senior director of clinical services housing ...,NaN,NaN,NaN,NaN,NaN
1,zs1YNrrCeorFkGLEXSSp-A_0000,[I believe that passions are meant to be share...,NaN,NaN,NaN,NaN,NaN
2,zowirq8ZhxhchApaRMhNSA_0000,[Bill Bryant is founder and chairman of Bryant...,NaN,NaN,NaN,NaN,NaN
3,zoQVkPCfXB9n3AaPGHHwzg_0000,[Undertaking a Trilingual masters degree in Eu...,NaN,NaN,NaN,NaN,NaN
4,zmIr2glBZ3Ef8CSS0jw1og_0000,[Career objective a role within marketing and ...,NaN,NaN,NaN,NaN,NaN


In [85]:
final_df = df_final.sort_values(by=['id'], ascending=False)

In [86]:
final_df.head()

,id,summaries,exact_matched_patt_contextual,Technology_Names,matched_NER_by_borders,matched_NER_final,NER_Extracted_Tags
0,zsDSYc5UzpyXekOABZBfnQ_0000,[Senior director of clinical services housing ...,NaN,NaN,NaN,NaN,NaN
1,zs1YNrrCeorFkGLEXSSp-A_0000,[I believe that passions are meant to be share...,NaN,NaN,NaN,NaN,NaN
2,zowirq8ZhxhchApaRMhNSA_0000,[Bill Bryant is founder and chairman of Bryant...,NaN,NaN,NaN,NaN,NaN
3,zoQVkPCfXB9n3AaPGHHwzg_0000,[Undertaking a Trilingual masters degree in Eu...,NaN,NaN,NaN,NaN,NaN
4,zmIr2glBZ3Ef8CSS0jw1og_0000,[Career objective a role within marketing and ...,NaN,NaN,NaN,NaN,NaN


In [87]:
final_df["matched_NER_by_borders"] = final_df["matched_NER_by_borders"].fillna(0)

In [88]:
# demo
for i in range(0, len(final_df)):
        if final_df["matched_NER_by_borders"][i] != 0:
            mylist = final_df["matched_NER_by_borders"][i]
            
            mylist = list(dict.fromkeys(mylist))
            final_df["matched_NER_by_borders"][i] = mylist
            

In [89]:
final_df.head()

,id,summaries,exact_matched_patt_contextual,Technology_Names,matched_NER_by_borders,matched_NER_final,NER_Extracted_Tags
0,zsDSYc5UzpyXekOABZBfnQ_0000,[Senior director of clinical services housing ...,NaN,NaN,0,NaN,NaN
1,zs1YNrrCeorFkGLEXSSp-A_0000,[I believe that passions are meant to be share...,NaN,NaN,0,NaN,NaN
2,zowirq8ZhxhchApaRMhNSA_0000,[Bill Bryant is founder and chairman of Bryant...,NaN,NaN,0,NaN,NaN
3,zoQVkPCfXB9n3AaPGHHwzg_0000,[Undertaking a Trilingual masters degree in Eu...,NaN,NaN,0,NaN,NaN
4,zmIr2glBZ3Ef8CSS0jw1og_0000,[Career objective a role within marketing and ...,NaN,NaN,0,NaN,NaN


In [90]:
final_df.to_csv("NER_tagged_1000_simple_spacy.csv", index = False)

# Until Now, it is simple spacy Run.

In [91]:
final_df.head(-1)

,id,summaries,exact_matched_patt_contextual,Technology_Names,matched_NER_by_borders,matched_NER_final,NER_Extracted_Tags
0,zsDSYc5UzpyXekOABZBfnQ_0000,[Senior director of clinical services housing ...,NaN,NaN,0,NaN,NaN
1,zs1YNrrCeorFkGLEXSSp-A_0000,[I believe that passions are meant to be share...,NaN,NaN,0,NaN,NaN
2,zowirq8ZhxhchApaRMhNSA_0000,[Bill Bryant is founder and chairman of Bryant...,NaN,NaN,0,NaN,NaN
3,zoQVkPCfXB9n3AaPGHHwzg_0000,[Undertaking a Trilingual masters degree in Eu...,NaN,NaN,0,NaN,NaN
4,zmIr2glBZ3Ef8CSS0jw1og_0000,[Career objective a role within marketing and ...,NaN,NaN,0,NaN,NaN
...,...,...,...,...,...,...,...
995,-JJetb2pA1gitMVdS2hxcQ_0000,[12 years of banking experience: 9 years withi...,NaN,NaN,0,NaN,NaN
996,-EQeYEq-03cWVCkfnsUgow_0000,"[Eric is an experienced risk, insurance, and p...",NaN,NaN,0,NaN,NaN
997,-EJPzKCOhrcLUKsIiC35Ow_0000,[Team manager with 8 years of experience enabl...,NaN,NaN,0,NaN,NaN
998,-BSYFSH4GWqbTH-JhxVExA_0000,[I am currently a junior at Cal Poly San Luis ...,NaN,NaN,0,NaN,NaN


In [93]:
# kmp_df = pd.read_csv("1000_tagged_simanchala/output_colab/KMP_string_Match_newer_1000_colab.csv")
kmp_df = pd.read_csv("1000_tagged_simanchala/output_local/KMP_string_Match_newer_1000_v2.csv")

In [99]:
final_df = final_df.sort_values(by=['id'], ascending=False)
final_df.reset_index(inplace = True, drop = True)

In [100]:
kmp_df = kmp_df.sort_values(by=['id'], ascending=False)
kmp_df.reset_index(inplace = True, drop = True)

In [101]:
kmp_df.head(2)

,Unnamed: 0,id,summaries,Technology,exact_matched_patt,matched_NER_final
0,0,zsDSYc5UzpyXekOABZBfnQ_0000,senior director of clinical services housing a...,"['ora', 'alis', 'and co', 'vserv', 'scons', 'e...",[],"['', '']"
1,1,zs1YNrrCeorFkGLEXSSp-A_0000,i believe that passions are meant to be shared...,"['xperience', 'ora', 'alis', 'and co', 'contin...",[],"['', '']"


In [102]:
type(kmp_df["exact_matched_patt"][1])

str

In [103]:
# Converting dumped data into proper format i.e string into list

# Pass the matched technology column that we got
def convert_data_format(df,column_name):
    
    for i in range(0, len(df)):
        
        if df[column_name][i][0] == '[':
            res = ast.literal_eval(df[column_name][i])
            df[column_name][i] = res
            
        else:
            x = []
            x.insert(0, df[column_name][i])
            df[column_name][i] = x
            
    return df

In [104]:
kmp_df = convert_data_format(kmp_df, "exact_matched_patt")

<ipython-input-103-b23b3f954057>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name][i] = res


In [105]:
type(kmp_df["exact_matched_patt"][0])

list

In [106]:
kmp_df["exact_matched_patt"][1]

[]

In [107]:
kmp_df["exact_matched_patt"][0]

[]

In [108]:
for i in range(0, len(kmp_df)):
    if len(kmp_df["exact_matched_patt"][i])==0:
        kmp_df["exact_matched_patt"][i] = ["0"]
        

<ipython-input-108-7f953530744a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kmp_df["exact_matched_patt"][i] = ["0"]


In [109]:
for i in range(0, len(final_df)):
    if (final_df["matched_NER_by_borders"][i])==0:
        final_df["matched_NER_by_borders"][i] = ["0"]
        

In [110]:
type(final_df["matched_NER_by_borders"][0])

list

In [111]:
for i in range(0,len(kmp_df)):
    print(i)
    if (kmp_df["id"][i]==final_df["id"][i]):
      
            final_df["matched_NER_by_borders"][i] = kmp_df["exact_matched_patt"][i]+final_df["matched_NER_by_borders"][i]            
            final_df["matched_NER_by_borders"][i] = list(set(final_df["matched_NER_by_borders"][i]))
        
#         final_df["matched_NER_by_borders"][i] = (final_df["matched_NER_by_borders"][i]+df_string_match["exact_matched_patt"][i])
#         final_df["matched_NER_by_borders"][i] = list(set(final_df["matched_NER_by_borders"][i]))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [112]:
final_df[final_df["id"]=="zaByWGsar5hVdBWLEYykcg_0000"]

,id,summaries,exact_matched_patt_contextual,Technology_Names,matched_NER_by_borders,matched_NER_final,NER_Extracted_Tags
8,zaByWGsar5hVdBWLEYykcg_0000,[As owner and director of projects at Metro In...,"['wordpress', 'joomla', 'drupal']",NaN,"[drupal, wordpress, 0, joomla]",NaN,NaN


In [113]:
# h = final_df["matched_NER_by_borders"][3].remove(0)

In [114]:
final_df["matched_NER_by_borders"][0][0]

'0'

In [115]:
for i in range(0, len(final_df)):
    l = final_df["matched_NER_by_borders"][i]
    final_df["matched_NER_by_borders"][i] = [0 if element == '0' else element for element in l]

In [116]:
# for i in range(0, len(final_df)):
#   l = final_df["matched_NER_by_borders"][i]
#   g = l[:]
#   l.remove(0)

In [117]:
for i in range(0, len(final_df)):
  words = final_df["matched_NER_by_borders"][i]
  stopwords = [0]
  for word in list(words):
    if word in stopwords:
      words.remove(word)

In [118]:
final_df

,id,summaries,exact_matched_patt_contextual,Technology_Names,matched_NER_by_borders,matched_NER_final,NER_Extracted_Tags
0,zsDSYc5UzpyXekOABZBfnQ_0000,[Senior director of clinical services housing ...,NaN,NaN,[],NaN,NaN
1,zs1YNrrCeorFkGLEXSSp-A_0000,[I believe that passions are meant to be share...,NaN,NaN,[],NaN,NaN
2,zowirq8ZhxhchApaRMhNSA_0000,[Bill Bryant is founder and chairman of Bryant...,NaN,NaN,[],NaN,NaN
3,zoQVkPCfXB9n3AaPGHHwzg_0000,[Undertaking a Trilingual masters degree in Eu...,NaN,NaN,[],NaN,NaN
4,zmIr2glBZ3Ef8CSS0jw1og_0000,[Career objective a role within marketing and ...,NaN,NaN,[],NaN,NaN
...,...,...,...,...,...,...,...
996,-EQeYEq-03cWVCkfnsUgow_0000,"[Eric is an experienced risk, insurance, and p...",NaN,NaN,[],NaN,NaN
997,-EJPzKCOhrcLUKsIiC35Ow_0000,[Team manager with 8 years of experience enabl...,NaN,NaN,[],NaN,NaN
998,-BSYFSH4GWqbTH-JhxVExA_0000,[I am currently a junior at Cal Poly San Luis ...,NaN,NaN,[astro],NaN,NaN
999,-4ldAQ53se0hp9GU5FgnIQ_0000,[An extremely driven professional within the B...,NaN,NaN,[],NaN,NaN


In [119]:
final_df[final_df["id"]=="zaByWGsar5hVdBWLEYykcg_0000"]

,id,summaries,exact_matched_patt_contextual,Technology_Names,matched_NER_by_borders,matched_NER_final,NER_Extracted_Tags
8,zaByWGsar5hVdBWLEYykcg_0000,[As owner and director of projects at Metro In...,"['wordpress', 'joomla', 'drupal']",NaN,"[drupal, wordpress, joomla]",NaN,NaN


In [120]:
r = pd.read_csv("Remove.csv")

In [121]:
for i in range(0,len(r)):
    r["Remove"][i] = r["Remove"][i].lower()

In [122]:
remove_list = r["Remove"].tolist()


In [123]:
remove_list.append("ve")


In [124]:
for i in range(0, len(final_df)):
    print(i)
    for word in list(final_df["matched_NER_by_borders"][i]):
        if word in remove_list:
            final_df["matched_NER_by_borders"][i].remove(word)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [125]:
final_df.to_csv("1000_tagged_simanchala/output_colab/final_df_string_ner_v4_1000_combined.csv", index = False)